In [1]:
from prf_api.prf_api import PRFApi
import pandas as pd

import re

In [2]:
data = PRFApi()

In [26]:
accidents_df = data.dataframe('acidentes_ocorrencia', estado='RN', caminho='../data/raw', 
                             anos=list(range(2007,2019)))

>>> Criando dataframe com ano 2007...


/home/mayra/workspace/corvusenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


>>> Criando dataframe com ano 2008...
>>> Criando dataframe com ano 2009...
>>> Criando dataframe com ano 2010...
>>> Criando dataframe com ano 2011...
>>> Criando dataframe com ano 2012...
>>> Criando dataframe com ano 2013...
>>> Criando dataframe com ano 2014...
>>> Criando dataframe com ano 2015...
>>> Criando dataframe com ano 2016...


/home/mayra/workspace/corvusenv/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


>>> Criando dataframe com ano 2017...
>>> Criando dataframe com ano 2018...


In [27]:
accidents_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35809 entries, 0 to 35808
Data columns (total 31 columns):
ano                       31313 non-null float64
br                        35804 non-null object
causa_acidente            35809 non-null object
classificacao_acidente    35809 non-null object
condicao_metereologica    35809 non-null object
data_inversa              35809 non-null object
delegacia                 2870 non-null object
dia_semana                35809 non-null object
fase_dia                  35809 non-null object
feridos                   35809 non-null int64
feridos_graves            35809 non-null int64
feridos_leves             35809 non-null int64
horario                   35809 non-null object
id                        35809 non-null float64
ignorados                 35809 non-null int64
ilesos                    35809 non-null int64
km                        35804 non-null object
latitude                  2870 non-null float64
longitude                 2870 

In [28]:
accidents_df.tail()

,ano,br,causa_acidente,classificacao_acidente,condicao_metereologica,data_inversa,delegacia,dia_semana,fase_dia,feridos,...,pessoas,regional,sentido_via,tipo_acidente,tipo_pista,tracado_via,uf,uop,uso_solo,veiculos
35804,NaN,304,Desobediência às normas de trânsito pelo condutor,Com Vítimas Feridas,Céu Claro,2018-09-22,DEL15/4,sábado,Plena Noite,2,...,2,SR-RN,Crescente,Colisão frontal,Dupla,Reta,RN,UOP02/RN,Sim,2
35805,NaN,101,Falta de Atenção à Condução,Com Vítimas Feridas,Sol,2018-12-26,DEL15/1,quarta-feira,Pleno dia,2,...,4,SR-RN,Crescente,Engavetamento,Dupla,Reta,RN,UOP01/RN,Sim,3
35806,NaN,101,Falta de Atenção à Condução,Com Vítimas Feridas,Garoa/Chuvisco,2018-01-15,DEL15/1,segunda-feira,Pleno dia,1,...,2,SR-RN,Crescente,Colisão lateral,Simples,Reta,RN,UOP01/RN,Sim,2
35807,NaN,101,Desobediência às normas de trânsito pelo condutor,Com Vítimas Feridas,Céu Claro,2018-12-16,DEL15/1,domingo,Pleno dia,1,...,2,SR-RN,Decrescente,Colisão traseira,Múltipla,Não Informado,RN,UOP01/RN,Sim,2
35808,NaN,304,Avarias e/ou desgaste excessivo no pneu,Com Vítimas Feridas,Ignorado,2018-12-07,DEL15/1,sexta-feira,Plena Noite,1,...,1,SR-RN,Crescente,Saída de leito carroçável,Simples,Reta,RN,UOP03/RN,Não,1


## Cleaning the data

The first step is to normalize the format of the date.

In [29]:
repl = lambda x: x.group(0)[-4:] + '-' + x.group(0)[-7:-5] + '-'+ x.group(0)[:2]
accidents_df['data'] = accidents_df.data_inversa.str.replace("[0-9]{2}/[0-9]{2}/[0-9]{4}",
                                                            repl, regex=True)

# Check out the result
print(accidents_df['data'].head())
print(accidents_df['data'].tail())

0    2007-01-01
1    2007-01-01
2    2007-01-01
3    2007-01-01
4    2007-01-01
Name: data, dtype: object
35804    2018-09-22
35805    2018-12-26
35806    2018-01-15
35807    2018-12-16
35808    2018-12-07
Name: data, dtype: object


Then, we drop the missing data in `km` and `br` columns.

In [30]:
accidents_df.dropna(subset=['km', 'br'], inplace=True)

Finally, the column `ano` (year) is correctly filled

In [31]:
accidents_df['ano'] = accidents_df.data.str.split('-').str.get(0)

accidents_df.ano.head()

0    2007
1    2007
2    2007
3    2007
4    2007
Name: ano, dtype: object

## Merging the datasets

In this section, we will aggregate the info about driving violations

In [32]:
# Initialize the new column
accidents_df['infracoes'] = 0

In [33]:
# Recover data
violations_df = pd.read_csv('../data/processed/violations_count.csv', sep=';',
                            names=['data','km','br', 'contagem'])

# Check out the data
violations_df.head()

,data,km,br,contagem
0,2015-01-01,0,427.0,0
1,2015-01-01,1,405.0,0
2,2015-01-01,1,427.0,0
3,2015-01-01,8,110.0,0
4,2015-01-01,9,110.0,0


In [34]:
violations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49098 entries, 0 to 49097
Data columns (total 4 columns):
data        49098 non-null object
km          49098 non-null int64
br          49098 non-null float64
contagem    49098 non-null int64
dtypes: float64(1), int64(2), object(1)
memory usage: 1.5+ MB


Now it's possible to update the number of driving violations based on the date of the violation and the point and road.

In [35]:
def filter_violations(row):
    data_value = row['data']
    km_value = float(str(row['km']).replace(',', '.'))
    br_value = float(row['br'])
    # Recover violation ate the same point
    violation = violations_df.query("data == '{}' and km == {} and br == {}".format(
                                      data_value,km_value, br_value))
    
    return len(violation['contagem']) if violation['contagem'].sum() else 0


accidents_df['infracoes'] = accidents_df.apply(filter_violations, axis='columns')

In [36]:
accidents_df.query('infracoes > 0')

,ano,br,causa_acidente,classificacao_acidente,condicao_metereologica,data_inversa,delegacia,dia_semana,fase_dia,feridos,...,sentido_via,tipo_acidente,tipo_pista,tracado_via,uf,uop,uso_solo,veiculos,data,infracoes
32940,2017,405,Ingestão de Álcool ...,Com Vítimas Fatais,Céu Claro,2017-01-01,DEL15/4,domingo,Pleno dia,0,...,Crescente,Colisão frontal ...,Simples,Reta,RN,UOP02/RN,Não,2,2017-01-01,1
32943,2017,304,Não guardar distância de segurança ...,Sem Vítimas,Céu Claro,2017-01-01,DEL15/4,domingo,Pleno dia,0,...,Crescente,Engavetamento ...,Simples,Reta,RN,UOP02/RN,Sim,3,2017-01-01,1
32946,2017,101,Falta de Atenção à Condução ...,Com Vítimas Feridas,Sol,2017-01-02,DEL15/1,segunda-feira,Pleno dia,1,...,Crescente,Colisão lateral ...,Dupla,Curva,RN,UOP01/RN,Sim,2,2017-01-02,1
32969,2017,226,Falta de Atenção à Condução ...,Com Vítimas Feridas,Céu Claro,2017-01-10,DEL15/1,terça-feira,Pleno dia,1,...,Crescente,Colisão frontal ...,Simples,Interseção de vias,RN,UOP02/RN,Não,2,2017-01-10,1
32970,2017,304,Pista Escorregadia ...,Com Vítimas Feridas,Garoa/Chuvisco,2017-01-10,DEL15/4,terça-feira,Pleno dia,1,...,Decrescente,Queda de ocupante de veículo ...,Dupla,Não Informado,RN,UOP01/RN,Sim,2,2017-01-10,1
32978,2017,304,Desobediência às normas de trânsito pelo condu...,Com Vítimas Feridas,Céu Claro,2017-01-12,DEL15/4,quinta-feira,Plena Noite,1,...,Crescente,Colisão frontal ...,Simples,Reta,RN,UOP02/RN,Não,2,2017-01-12,1
32979,2017,304,Não guardar distância de segurança ...,Sem Vítimas,Céu Claro,2017-01-12,DEL15/1,quinta-feira,Plena Noite,0,...,Crescente,Colisão traseira ...,Simples,Reta,RN,UOP02/RN,Sim,2,2017-01-12,1
32980,2017,304,Desobediência às normas de trânsito pelo condu...,Com Vítimas Feridas,Sol,2017-01-13,DEL15/1,sexta-feira,Pleno dia,1,...,Crescente,Colisão transversal ...,Simples,Interseção de vias,RN,UOP02/RN,Sim,2,2017-01-13,1
32981,2017,304,Desobediência às normas de trânsito pelo condu...,Com Vítimas Feridas,Céu Claro,2017-01-13,DEL15/4,sexta-feira,Pleno dia,3,...,Crescente,Colisão transversal ...,Dupla,Retorno Regulamentado,RN,UOP01/RN,Sim,2,2017-01-13,1
32983,2017,304,Desobediência às normas de trânsito pelo condu...,Sem Vítimas,Céu Claro,2017-01-13,DEL15/4,sexta-feira,Pleno dia,0,...,Decrescente,Colisão lateral ...,Simples,Rotatória,RN,UOP02/RN,Sim,2,2017-01-13,1


In [37]:
accidents_df.to_csv('../data/processed/accidents.csv', sep=';')